<a href="https://colab.research.google.com/github/duandaxia/open-pi-zero/blob/main/fast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers

## Local Inference on GPU
Model page: https://huggingface.co/physical-intelligence/fast

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/physical-intelligence/fast)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [1]:
# Load model directly
from transformers import AutoProcessor
model = AutoProcessor.from_pretrained("physical-intelligence/fast",trust_remote_code=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/253 [00:00<?, ?B/s]

processing_action_tokenizer.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/physical-intelligence/fast:
- processing_action_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/322 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

In [2]:
import numpy as np

action = np.array([ [ 0.09642857, -0.02410714, -0.,          0.,          0.,         -0.,
  -1.        ],
 [ 0.13928571, -0.03214286, -0.,          0.,          0.,         -0.,
  -1.        ],
 [ 0.3375,      0. ,        -0.    ,     -0.00964286,  0.    ,     -0.,
  -1.        ],
 [ 0.50357143,  0.  ,        0.02410714, -0.02142857,  0.00107143, -0.,
  -1.        ],
 [ 0.68035714,  0.  ,        0.01875,    -0.02571429,  0.03214286, -0.,
  -1.        ],
 [ 0.73392857,  0.04285714,  0.01875,    -0.01714286,  0.04392857,   -0.,
  -1.        ],
 [ 0.84642857,  0.08839286,  0.04285714, -0.01392857,  0.0525,     -0.,
  -1.        ],
 [ 0.89196429,  0.14196429,  0.03482143, -0.01392857,  0.05035714, -0.,
  -1.        ],
 [ 0.9375,      0.23035714, -0.,         -0.01607143,  0.04928571, -0.,
  -1.        ],
 [ 0.9375,      0.31875,    -0.,         -0.01392857,  0.05464286,  0.00642857,
  -1.        ]
])




In [10]:
import numpy as np

normalized_action_scaled = np.zeros_like(action, dtype=float)

for i in range(action.shape[1]): # Iterate over each dimension (column)
    col_data = action[:, i]

    # Create a mask for non-zero elements in the current column
    non_zero_mask = (col_data != 0)
    non_zero_values = col_data[non_zero_mask]

    if len(non_zero_values) > 1: # Need at least 2 distinct values for meaningful min/max
        min_val = np.min(non_zero_values)
        max_val = np.max(non_zero_values)

        diff = max_val - min_val

        if diff == 0: # Handle cases where all non-zero values are the same
            normalized_action_scaled[non_zero_mask, i] = 0.0
        else:
            # Apply min-max scaling to non-zero values only, mapping to [-1, 1]
            normalized_action_scaled[non_zero_mask, i] = -1 + 2 * (non_zero_values - min_val) / diff
    elif len(non_zero_values) == 1:
        # If there's only one unique non-zero value, map it to 0 in the normalized range
        normalized_action_scaled[non_zero_mask, i] = 0.0
    # If len(non_zero_values) == 0, all values in the column are zero,
    # and they remain zero due to normalized_action_scaled initialization.

print("Original action array:")
print(action)
print("\nNormalized action array (min-max scaled to [-1, 1] for non-zeros by column):")
print(normalized_action_scaled)

Original action array:
[[ 0.09642857 -0.02410714 -0.          0.          0.         -0.
  -1.        ]
 [ 0.13928571 -0.03214286 -0.          0.          0.         -0.
  -1.        ]
 [ 0.3375      0.         -0.         -0.00964286  0.         -0.
  -1.        ]
 [ 0.50357143  0.          0.02410714 -0.02142857  0.00107143 -0.
  -1.        ]
 [ 0.68035714  0.          0.01875    -0.02571429  0.03214286 -0.
  -1.        ]
 [ 0.73392857  0.04285714  0.01875    -0.01714286  0.04392857 -0.
  -1.        ]
 [ 0.84642857  0.08839286  0.04285714 -0.01392857  0.0525     -0.
  -1.        ]
 [ 0.89196429  0.14196429  0.03482143 -0.01392857  0.05035714 -0.
  -1.        ]
 [ 0.9375      0.23035714 -0.         -0.01607143  0.04928571 -0.
  -1.        ]
 [ 0.9375      0.31875    -0.         -0.01392857  0.05464286  0.00642857
  -1.        ]]

Normalized action array (min-max scaled to [-1, 1] for non-zeros by column):
[[-1.         -0.95419844  0.          0.          0.          0.
   0.        ]

In [6]:
import numpy as np

normalized_action_filtered = np.zeros_like(action, dtype=float)

for i in range(action.shape[1]): # Iterate over each dimension (column)
    col_data = action[:, i]

    # Create a mask for non-zero elements in the current column
    non_zero_mask = (col_data != 0)
    non_zero_values = col_data[non_zero_mask]

    if len(non_zero_values) > 1: # Need at least 2 distinct values to calculate 1st and 99th percentile meaningfully
        q1 = np.percentile(non_zero_values, 1)
        q99 = np.percentile(non_zero_values, 99)

        diff = q99 - q1

        if diff == 0: # Handle cases where all non-zero values are the same
            normalized_action_filtered[non_zero_mask, i] = 0.0
        else:
            # Apply normalization to non-zero values only
            normalized_action_filtered[non_zero_mask, i] = -1 + 2 * (non_zero_values - q1) / diff
    elif len(non_zero_values) == 1:
        # If there's only one unique non-zero value, map it to 0 in the normalized range
        normalized_action_filtered[non_zero_mask, i] = 0.0
    # If len(non_zero_values) == 0, all values in the column are zero,
    # and they remain zero due to normalized_action_filtered initialization.

print("Original action array:")
print(action)
print("\nNormalized action array (non-zero based quantiles, zeros preserved) by column:")
print(normalized_action_filtered)

Original action array:
[[ 0.09642857 -0.02410714 -0.          0.          0.         -0.
  -1.        ]
 [ 0.13928571 -0.03214286 -0.          0.          0.         -0.
  -1.        ]
 [ 0.3375      0.         -0.         -0.00964286  0.         -0.
  -1.        ]
 [ 0.50357143  0.          0.02410714 -0.02142857  0.00107143 -0.
  -1.        ]
 [ 0.68035714  0.          0.01875    -0.02571429  0.03214286 -0.
  -1.        ]
 [ 0.73392857  0.04285714  0.01875    -0.01714286  0.04392857 -0.
  -1.        ]
 [ 0.84642857  0.08839286  0.04285714 -0.01392857  0.0525     -0.
  -1.        ]
 [ 0.89196429  0.14196429  0.03482143 -0.01392857  0.05035714 -0.
  -1.        ]
 [ 0.9375      0.23035714 -0.         -0.01607143  0.04928571 -0.
  -1.        ]
 [ 0.9375      0.31875    -0.         -0.01392857  0.05464286  0.00642857
  -1.        ]]

Normalized action array (non-zero based quantiles, zeros preserved) by column:
[[-1.00921423 -0.95622474  0.          0.          0.          0.
   0.       

In [ ]:
norm_token = model()

In [ ]:
norm_token = model(normalized_action_filtered)
norm_decode = model.decode(norm_token)
print(np.lin)